In [ ]:
# Team Members:

# Shubh Almal 
# ID: 801365752

# Saakshi Patel 
# ID: 801361041

In [ ]:
# 2c. How's the policy of states? 

In [ ]:
# Block 1: Importing Libraries

# This block imports the required libraries, including NetworkX for creating and analyzing networks, Matplotlib for basic plotting 
# functionalities, and Plotly for interactive graph visualizations.

In [12]:
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
# Block 2: Creating the Network Graph

# This block creates a directed graph and adds nodes for each policy term. It then calculates the co-occurrence of terms in the 
# policy text and adds edges between terms based on their co-occurrence.

In [13]:
# Create a directed graph
G = nx.DiGraph()

# Add nodes for each policy term
for term in policy_terms:
    G.add_node(term)

# Calculate co-occurrence of terms
co_occurrence_matrix = pd.DataFrame(0, index=policy_terms, columns=policy_terms)
for sentence in policy_text.split('.'):
    for term1 in policy_terms:
        if term1 in sentence:
            for term2 in policy_terms:
                if term2 in sentence and term1 != term2:
                    co_occurrence_matrix.loc[term1, term2] += 1

# Add edges based on co-occurrence
for term1 in policy_terms:
    for term2 in policy_terms:
        if co_occurrence_matrix.loc[term1, term2] > 0:
            G.add_edge(term1, term2, weight=co_occurrence_matrix.loc[term1, term2])


In [ ]:
# Block 3: Visualizing the Network Graph

# This block calculates positions for the nodes using the spring layout algorithm, creates edge and node traces for Plotly visualization, 
# and creates the network graph figure with interactive features such as hover information and node connections.

In [14]:
# Calculate positions for the nodes
pos = nx.spring_layout(G, seed=42)

# Create edge traces
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Create node traces
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

# Add positions to node and edge traces
for node in G.nodes():
    x, y = pos[node]
    node_trace['x'] += (x,)
    node_trace['y'] += (y,)
    node_trace['text'] += (node,)

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# Create network graph figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Network Graph of Policy Terms Co-occurrence',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        text="",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002)],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

# Show the network graph
fig.show()


In [ ]:
# Conclusion and Insight

# The visualization presents a network graph showing the co-occurrence of policy terms. Nodes represent policy terms, and edges 
# between nodes indicate how often those terms appear together in the policy text. This visualization helps in understanding the 
# relationships and associations between different policy terms, providing insights into the thematic structure or connections within 
# the policy documents.